In [20]:
from ultralytics import YOLO
import os
from collections import Counter
import pandas as pd
# Define the paths to the configuration files and dataset
data_yaml_path = '/home/subbu/projects/hackathon_unx_ord/data.yaml'  # Dataset configuration file

In [ ]:

# Define the paths to the configuration files and dataset
# model = YOLO('yolov8n.pt')
# results = model.train(
#     data=data_yaml_path,
#     imgsz=640,
#     epochs=25,
#     batch=4)


In [4]:
# Path where the trained model is saved
model_path = './output/runs/detect/train3222/weights/best.pt'

# Load the trained YOLO model
model = YOLO(model_path)

In [7]:
results = model(['/home/subbu/projects/hackathon_unx_ord/data/UXOTi_NPA_11_class/Novi11/train/images/frame_000683.PNG'])  # return a list of Results objects
results[0].save('result.jpg')

0: 448x640 1 Mortar mine M: 60 mm, 1 Fuse M: 125 mm, 48.8ms
Speed: 3.0ms preprocess, 48.8ms inference, 0.4ms postprocess per image at shape (1, 3, 448, 640)


'result.jpg'

In [18]:
class_names = {
    0: 'Mortar mine M: 60 mm',
    1: 'Fuse M: 125 mm',
    2: 'Hand grenade',
    3: 'AP land mine PMR capjinka',
    4: 'Bullet 30.2 mm',
    5: 'Land mine PMR 2A',
    6: 'Mortar mine M: 82 mm',
    7: 'Land mine PMR 3',
    8: 'Land mine PROM 1',
    9: 'Land mine PMA 3',
    10: 'Tromblon mine TTM RP'
}

In [25]:
def count_classes_in_partition(label_dir):
    counter = Counter()
    for label_file in os.listdir(label_dir):
        if label_file.endswith('.txt') and not label_file.startswith('classes'):
            with open(os.path.join(label_dir, label_file), 'r') as file:
                for line in file:
                    class_id = int(line.split()[0])
                    counter[class_id] += 1
    return counter

train_labels_path = './data/UXOTi_NPA_11_class/Novi11/train/labels/'
val_labels_path = './data/UXOTi_NPA_11_class/Novi11/valid/labels/'
test_labels_path = './data/UXOTi_NPA_11_class/Novi11/test/labels/'

train_counter = count_classes_in_partition(train_labels_path)
val_counter = count_classes_in_partition(val_labels_path)
test_counter = count_classes_in_partition(test_labels_path)

data = []
for class_id, name in class_names.items():
    row = {
        'Class ID': class_id,
        'Class Name': name,
        'Train': train_counter[class_id],
        'Validation': val_counter[class_id],
        'Test': test_counter[class_id]
    }
    data.append(row)

df = pd.DataFrame(data)
df['Total'] = df['Train'] + df['Validation'] + df['Test']

# Check for any class that is missing in any partition
all_classes = set(train_counter) | set(val_counter) | set(test_counter)
print("All classes present:", all_classes == set(train_counter) == set(val_counter) == set(test_counter))

df

All classes present: True


,Class ID,Class Name,Train,Validation,Test,Total
0,0,Mortar mine M: 60 mm,139,15,12,166
1,1,Fuse M: 125 mm,161,15,11,187
2,2,Hand grenade,161,18,10,189
3,3,AP land mine PMR capjinka,121,19,14,154
4,4,Bullet 30.2 mm,104,16,15,135
5,5,Land mine PMR 2A,116,13,16,145
6,6,Mortar mine M: 82 mm,136,20,31,187
7,7,Land mine PMR 3,149,26,40,215
8,8,Land mine PROM 1,150,23,27,200
9,9,Land mine PMA 3,139,15,13,167
